# Advent of Code 2020 - Day 7

In [101]:
@JvmInline
value class Bag(val name: String)
data class Node(val bag: Bag, val edges: MutableList<Edge> = mutableListOf())
data class Edge(val amount: Int, val node: Node)

class Graph {
    private val _nodes: MutableMap<Bag, Node> = mutableMapOf()
    val nodes: Map<Bag, Node> = _nodes

    fun put(bag: Bag, contains: List<Pair<Int, Bag>>): Node = 
        _nodes
            .getOrPut(bag) { Node(bag = bag) }
            .also {
                it.edges.addAll(
                    contains.map { 
                        Edge(
                            amount = it.first,
                            node = _nodes.getOrPut(it.second) { Node(it.second) }
                        )
                    }
                )
            }
}

In [102]:
import java.io.File

val graph = Graph()

val rules = File("Day7.input.txt")
    .bufferedReader()
    .readLines()
    .map { line -> Regex("""(^\w+ \w+|\d \w+ \w+)""").findAll(line).flatMap { it.destructured.toList() }.toList() }
    .map { rules: List<String> ->
        graph.put(
            bag = Bag(rules[0]),
            contains = rules.drop(1).map {
                val (amount, bag) = Regex("""(\d+) (\w+ \w+)""").find(it)!!.destructured
                amount.toInt() to Bag(bag)
            }
        )
    }

### Notes

The regex here is kind of clever, once you realize that the input format is either two words, or a digit and two words, parsing doesn't become too difficult. The input data is pretty gross though.

Recognized that the relationships were a graph, but not super happy with the graph API. It works, so rather than fiddling with it too much it'll just stay like this for now.

## Part 1

How many bags will _eventually_ contain a `shiny gold` bag?

In [103]:
fun Node.contains(target: Bag): Boolean = bag == target || edges.any { it.node.contains(target) }

Bag("shiny gold").let { target: Bag ->
    graph.nodes.values
        .filter { it.bag != target }
        .count { it.contains(target) }
}

205

### Notes

This could be solved pretty straightforwardly with either a BFS or a DFS. In my case, I implemented it as a DFS since I felt like `any` made logical sense with `contains`. The hardest part here was probably building the graph.

## Part 2

How many bags are required inside of a `shiny gold` bag?

In [104]:
fun Node.countBags(): Int = edges.sumOf { it.amount + it.amount * it.node.countBags() }

graph.nodes[Bag("shiny gold")]?.countBags()

80902

### Notes

Another situation of if the graph is setup properly, then everything else is easy. Just needed to traverse all the nodes and sum/multiply them up. This could probably be improved by making it `tailrec`.